<a href="https://colab.research.google.com/github/carlosdgerez/machine_learning/blob/main/module3Boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the basic imports and dataset

In [1]:
import pandas as pd

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
housing.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,1565930130,20141104T000000,4,3.25,3760,4675,2.0,0,0,3,...,2740,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0
1,3279000420,20150115T000000,3,1.75,1460,7800,1.0,0,0,2,...,1040,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0
2,194000575,20141014T000000,4,1.00,1340,5800,1.5,0,2,3,...,1340,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0
3,2115510160,20141208T000000,3,1.75,1440,8050,1.0,0,0,3,...,1440,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0
4,7522500005,20140815T000000,2,1.50,1780,4750,1.0,0,0,4,...,1080,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0


All from here are extra information added to the original dataset and some transformations to the dates( years )

In [2]:
# download the income data

zip_code = pd.read_csv('https://raw.githubusercontent.com/carlosdgerez/machine_learning/main/mean_income_washington_by_zipcode.csv')
zip_code.head()



,zip_code,mean_income
0,98001,102586
1,98002,70945
2,98003,76753
3,98004,210129
4,98005,186020


In [3]:
# rename the columns as the one in clean data to use merge by this column
zip_code.rename(columns={'zip_code': 'zipcode'}, inplace = True)

In [4]:
# more rich data that improve the model from Kate

zipcode = pd.read_csv('https://raw.githubusercontent.com/Kate-Strydom/cse450/main/notebooks/zipcode_washington.csv')

zipcode.head()

,zipcode,population,median_household_income
0,98001,31742,72540
1,98002,34240,44930
2,98003,45632,45563
3,98004,31254,100344
4,98005,17880,85639


In [5]:
#Changing date in the original set to be in the same format as the rest of the years features
import datetime
housing['date'] = pd.DatetimeIndex(housing['date']).year



In [6]:
# drop id
# droping the id from the original dataset since can be related to personal information start to generate a set to work with
clean_data = housing.drop(['id'], axis = 1)

In [7]:

# change date name to something more descriptive
clean_data.rename(columns={'date': 'sale_date'}, inplace=True)

In [8]:
# Use merge to get the data in the dataframe
new_clean_data = pd.merge(clean_data, zip_code, how= 'left', on= 'zipcode')
new_clean_data.head()

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price,mean_income
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,1020,2007,0,98038,47.3862,-122.048,3280,4033,429900.0,134178
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,420,1979,0,98023,47.3035,-122.382,1310,7865,233000.0,100910
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,1914,0,98116,47.5658,-122.389,1900,5800,455000.0,151980
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,1985,0,98023,47.3187,-122.390,1790,7488,258950.0,100910
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,700,1947,0,98117,47.6859,-122.395,1690,5962,555000.0,166123


In [9]:
# This reorder can be done better in the final set, but since I worked in progression is a remainder of the original work that can be usefoul to test diferent sets in this same code.
# reorder the data to have the target price at the end
column_to_move = new_clean_data.pop("mean_income")

# insert column with insert(location, column_name, column_value)

new_clean_data.insert(19, "mean_income", column_to_move)


In [10]:
new_clean_data

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,price
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,1020,2007,0,98038,47.3862,-122.048,3280,4033,134178,429900.0
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,420,1979,0,98023,47.3035,-122.382,1310,7865,100910,233000.0
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,1914,0,98116,47.5658,-122.389,1900,5800,151980,455000.0
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,1985,0,98023,47.3187,-122.390,1790,7488,100910,258950.0
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,700,1947,0,98117,47.6859,-122.395,1690,5962,166123,555000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2014,3,1.50,1000,6914,1.0,0,0,3,7,...,0,1947,0,98125,47.7144,-122.319,1000,6947,103336,378000.0
19996,2014,3,2.50,3087,5002,2.0,0,0,3,8,...,0,2014,0,98023,47.2974,-122.349,2927,5183,100910,399950.0
19997,2014,3,2.50,2120,4780,2.0,0,0,3,7,...,0,2004,0,98053,47.6810,-122.032,1690,2650,205610,575000.0
19998,2014,1,0.75,380,15000,1.0,0,0,3,5,...,0,1963,0,98168,47.4810,-122.323,1170,15000,83229,245000.0


In [11]:
# Use merge to get the data in the dataframe
new_clean_data2 = pd.merge(new_clean_data, zipcode, how= 'left', on= 'zipcode')
new_clean_data2

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,price,population,median_household_income
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,0,98038,47.3862,-122.048,3280,4033,134178,429900.0,34076,93573
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,0,98023,47.3035,-122.382,1310,7865,100910,233000.0,49402,67881
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,98116,47.5658,-122.389,1900,5800,151980,455000.0,23780,73667
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,98023,47.3187,-122.390,1790,7488,100910,258950.0,49402,67881
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,0,98117,47.6859,-122.395,1690,5962,166123,555000.0,31778,91199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2014,3,1.50,1000,6914,1.0,0,0,3,7,...,0,98125,47.7144,-122.319,1000,6947,103336,378000.0,37560,52353
19996,2014,3,2.50,3087,5002,2.0,0,0,3,8,...,0,98023,47.2974,-122.349,2927,5183,100910,399950.0,49402,67881
19997,2014,3,2.50,2120,4780,2.0,0,0,3,7,...,0,98053,47.6810,-122.032,1690,2650,205610,575000.0,20739,119637
19998,2014,1,0.75,380,15000,1.0,0,0,3,5,...,0,98168,47.4810,-122.323,1170,15000,83229,245000.0,33604,50054


In [12]:
# This reorder can be done better in the final set, but since I worked in progression is a remainder of the original work that can be usefoul to test diferent sets in this same code.
# reorder the data to have the target price at the end
column_to_move = new_clean_data2.pop("price")

# insert column with insert(location, column_name, column_value)

new_clean_data2.insert(22, "price", column_to_move)


In [13]:
new_clean_data2

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,population,median_household_income,price
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,0,98038,47.3862,-122.048,3280,4033,134178,34076,93573,429900.0
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,0,98023,47.3035,-122.382,1310,7865,100910,49402,67881,233000.0
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,98116,47.5658,-122.389,1900,5800,151980,23780,73667,455000.0
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,98023,47.3187,-122.390,1790,7488,100910,49402,67881,258950.0
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,0,98117,47.6859,-122.395,1690,5962,166123,31778,91199,555000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2014,3,1.50,1000,6914,1.0,0,0,3,7,...,0,98125,47.7144,-122.319,1000,6947,103336,37560,52353,378000.0
19996,2014,3,2.50,3087,5002,2.0,0,0,3,8,...,0,98023,47.2974,-122.349,2927,5183,100910,49402,67881,399950.0
19997,2014,3,2.50,2120,4780,2.0,0,0,3,7,...,0,98053,47.6810,-122.032,1690,2650,205610,20739,119637,575000.0
19998,2014,1,0.75,380,15000,1.0,0,0,3,5,...,0,98168,47.4810,-122.323,1170,15000,83229,33604,50054,245000.0


From now down we start to check the models. The best parameters came fro cross validation in other code. We can test the original data (new_clean_data, and step by step the increase of performance adding more informatio by changing the code here. Instead of new_clean_data set new_clean_data2 or 3.

In [14]:
# get a training set: a X as with the features to relate and a y with the  target 
X = new_clean_data2.iloc[:,0:-1]
y = new_clean_data2.price


In [15]:
X

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,population,median_household_income
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,2007,0,98038,47.3862,-122.048,3280,4033,134178,34076,93573
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,1979,0,98023,47.3035,-122.382,1310,7865,100910,49402,67881
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,1914,0,98116,47.5658,-122.389,1900,5800,151980,23780,73667
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,1985,0,98023,47.3187,-122.390,1790,7488,100910,49402,67881
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,1947,0,98117,47.6859,-122.395,1690,5962,166123,31778,91199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2014,3,1.50,1000,6914,1.0,0,0,3,7,...,1947,0,98125,47.7144,-122.319,1000,6947,103336,37560,52353
19996,2014,3,2.50,3087,5002,2.0,0,0,3,8,...,2014,0,98023,47.2974,-122.349,2927,5183,100910,49402,67881
19997,2014,3,2.50,2120,4780,2.0,0,0,3,7,...,2004,0,98053,47.6810,-122.032,1690,2650,205610,20739,119637
19998,2014,1,0.75,380,15000,1.0,0,0,3,5,...,1963,0,98168,47.4810,-122.323,1170,15000,83229,33604,50054


In [16]:
# Start spliting the data and import libraries

#import train split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.30, random_state=1)


In [17]:
#  xgboost regresor
import xgboost
from sklearn import metrics as sm

# createn instance and fit the model

clf2 = xgboost.XGBRegressor(random_state=5, max_depth= 4, n_estimators= 900, learning_rate= 0.1, verbosity = 0, n_jobs = -1).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

# This is how to generate a regresion report ( for regresion )
print("Mean absolute error in dollars=", round(
    sm.mean_absolute_error(y_test, y_pred), 2))
print("Mean squared error =", round(
    sm.mean_squared_error(y_test, y_pred), 2))
print("Median absolute error =", round(
    sm.median_absolute_error(y_test, y_pred), 2))
print("Explain variance score =", round(
    sm.explained_variance_score(y_test, y_pred), 2))
print("R2 score =", round( sm.r2_score(y_test, y_pred),4))
print( "ROOT Mean squared error in dollars = ", 
    round(sm.mean_squared_error(y_test, y_pred, squared=False 
)))


Mean absolute error in dollars= 61911.21
Mean squared error = 11884782245.04
Median absolute error = 36993.66
Explain variance score = 0.91
R2 score = 0.9109
ROOT Mean squared error in dollars =  109017


The diference between the mean absolute result and the root squared may be for the high variation of the target values. Mean absolute error is less sensitive to the outliers in the performance scoring.

In [18]:


#  EVALUATION OF THE MODEL 

#Predict the response for test dataset


# Features importances in a model this block can be run with diferent models to compare


feat_imports = (pd.DataFrame({"feature names": X_train.columns,
                              "importances": clf2.feature_importances_})
                .sort_values("importances", ascending=False))

feat_imports.reset_index(drop= True, inplace= True)



In [19]:
# This presentation is basic, can be improved

import altair as alt
alt.themes.enable('fivethirtyeight')
feat_imports['percents'] = round((feat_imports.importances * 100).astype(float),2)
alt.Chart(feat_imports.head(10)).mark_bar().encode(
    alt.X("importances", axis=alt.Axis(format='%', labelColor='#D35400', titleColor='#af3f38' ), title='Percent of relevance in the model'),
    alt.Y('feature names',sort='-x', title= '',axis=alt.Axis(labelColor='#af3f38')),
    tooltip=('percents')).properties(title= 'Wich features are more relevant in price predictions.').configure_title(
        color='#ff998a'
    )

alt.Chart(...)

In [20]:
# Cross validation of the model with r2 as scoring parameter. Takes at least 1 minute to run all cicles.
from sklearn.model_selection import cross_val_score


scores = cross_val_score(clf2, X, y, scoring='r2', cv=10, n_jobs=-1)
scores.mean()

0.9017964559035805

In [21]:
scores

array([0.89105787, 0.87251651, 0.91508486, 0.90902371, 0.89509317,
       0.91009678, 0.92123603, 0.90110821, 0.90374584, 0.89900158])

# Holdout set predictions and saved model

Preprocessing ( has to repeat all the process of preparation of the dataset)
clf2 is the model to use.

In [22]:


housing2 = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing_holdout_test.csv')
housing2.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [23]:
#Changing date in the original set to be in the same format as the rest of the years features

housing2['date'] = pd.DatetimeIndex(housing2['date']).year



In [24]:
# drop id
# droping the id from the original dataset since can be related to personal information start to generate a set to work with
clean_data = housing2.drop(['id'], axis = 1)

In [25]:

# change date name to something more descriptive
clean_data.rename(columns={'date': 'sale_date'}, inplace=True)

In [26]:
# Use merge to get the data in the dataframe
new_clean_data = pd.merge(clean_data, zip_code, how= 'left', on= 'zipcode')
new_clean_data.head()

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income
0,2014,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700,106748
1,2014,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283,154113
2,2014,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685,198704
3,2015,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226,127177
4,2014,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050,165241


In [27]:
# Use merge to get the data in the dataframe
new_clean_data3 = pd.merge(new_clean_data, zipcode, how= 'left', on= 'zipcode')

In [28]:
new_clean_data3.columns

Index(['sale_date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'mean_income', 'population',
       'median_household_income'],
      dtype='object')

In [29]:
X = new_clean_data3

In [30]:
X

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,population,median_household_income
0,2014,4,2.25,2070,8893,2.0,0,0,4,8,...,1986,0,98058,47.4388,-122.162,2390,7700,106748,42764,81221
1,2014,5,3.00,2900,6730,1.0,0,0,5,8,...,1977,0,98115,47.6784,-122.285,2370,6283,154113,48558,88633
2,2014,4,2.50,3770,10893,2.0,0,2,3,11,...,1997,0,98006,47.5646,-122.129,3710,9685,198704,37059,111216
3,2015,3,3.50,4560,14608,2.0,0,2,3,12,...,1990,0,98034,47.6995,-122.228,4050,14226,127177,42335,81037
4,2014,3,2.50,2550,5376,2.0,0,0,3,9,...,2004,0,98052,47.6647,-122.083,2250,4050,165241,60920,103086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,2015,3,1.75,2160,5760,1.0,0,0,4,8,...,1954,0,98136,47.5346,-122.391,2090,5760,147330,15302,85040
1609,2014,4,2.50,2220,4228,2.0,0,0,3,8,...,2014,0,98146,47.5014,-122.341,2220,4157,98981,25722,56320
1610,2015,3,1.75,1390,8980,1.0,0,0,4,8,...,1985,0,98022,47.2015,-122.005,1770,9085,112118,21353,65864
1611,2014,3,2.25,2930,6000,2.0,0,2,3,11,...,2000,0,98112,47.6301,-122.301,1870,5040,216581,22290,100694


In [31]:
#  get the predictions

predictions = clf2.predict(X)




In [32]:
# convert to dataframe to see results
predictions = pd.DataFrame(predictions)
predictions

,0
0,3.903851e+05
1,8.679389e+05
2,1.096476e+06
3,2.038264e+06
4,9.110075e+05
...,...
1608,6.449515e+05
1609,4.447570e+05
1610,2.766465e+05
1611,1.353178e+06


In [33]:
#rename the column 0

predictions.rename(columns={0 : 'predictions'}, inplace= True)

# create a csv file in colab without index

predictions.to_csv('predictions.csv', index = False)

# download the file to the user computer current directory
from google.colab import files

files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
# Persistance of the model

import pickle
sampleModel = pickle.dumps(clf2, pickle.HIGHEST_PROTOCOL)
clf2 = pickle.loads(sampleModel)
clf2.predict(X_test)

array([569178.8 , 502189.8 , 340819.66, ..., 211808.31, 253107.66,
       675610.25], dtype=float32)

In [35]:

y_pred = clf2.predict(X_test)

# This is how to generate a regresion report ( for regresion )
print("Mean absolute error in dollars=", round(
    sm.mean_absolute_error(y_test, y_pred), 2))
print("Mean squared error =", round(
    sm.mean_squared_error(y_test, y_pred), 2))
print("Median absolute error =", round(
    sm.median_absolute_error(y_test, y_pred), 2))
print("Explain variance score =", round(
    sm.explained_variance_score(y_test, y_pred), 2))
print("R2 score =", round( sm.r2_score(y_test, y_pred),4))
print( "ROOT Mean squared error in dollars = ", 
    round(sm.mean_squared_error(y_test, y_pred, squared=False 
)))


Mean absolute error in dollars= 61911.21
Mean squared error = 11884782245.04
Median absolute error = 36993.66
Explain variance score = 0.91
R2 score = 0.9109
ROOT Mean squared error in dollars =  109017


# Try a new model with encoded zip codes 

In [42]:
new_clean_data2

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,mean_income,population,median_household_income,price
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,0,98038,47.3862,-122.048,3280,4033,134178,34076,93573,429900.0
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,0,98023,47.3035,-122.382,1310,7865,100910,49402,67881,233000.0
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,98116,47.5658,-122.389,1900,5800,151980,23780,73667,455000.0
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,98023,47.3187,-122.390,1790,7488,100910,49402,67881,258950.0
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,0,98117,47.6859,-122.395,1690,5962,166123,31778,91199,555000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2014,3,1.50,1000,6914,1.0,0,0,3,7,...,0,98125,47.7144,-122.319,1000,6947,103336,37560,52353,378000.0
19996,2014,3,2.50,3087,5002,2.0,0,0,3,8,...,0,98023,47.2974,-122.349,2927,5183,100910,49402,67881,399950.0
19997,2014,3,2.50,2120,4780,2.0,0,0,3,7,...,0,98053,47.6810,-122.032,1690,2650,205610,20739,119637,575000.0
19998,2014,1,0.75,380,15000,1.0,0,0,3,5,...,0,98168,47.4810,-122.323,1170,15000,83229,33604,50054,245000.0


In [43]:
new_clean_data4 =  pd.get_dummies(new_clean_data2, columns = ['zipcode'])



In [44]:
new_clean_data4

,sale_date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode_98146,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199
0,2014,4,3.25,3760,4675,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
1,2015,3,1.75,1460,7800,1.0,0,0,2,7,...,0,0,0,0,0,0,0,0,0,0
2,2014,4,1.00,1340,5800,1.5,0,2,3,7,...,0,0,0,0,0,0,0,0,0,0
3,2014,3,1.75,1440,8050,1.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
4,2014,2,1.50,1780,4750,1.0,0,0,4,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2014,3,1.50,1000,6914,1.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
19996,2014,3,2.50,3087,5002,2.0,0,0,3,8,...,0,0,0,0,0,0,0,0,0,0
19997,2014,3,2.50,2120,4780,2.0,0,0,3,7,...,0,0,0,0,0,0,0,0,0,0
19998,2014,1,0.75,380,15000,1.0,0,0,3,5,...,0,0,0,0,1,0,0,0,0,0


In [47]:
new_clean_data4.columns

Index(['sale_date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'mean_income', 'population',
       'median_household_income', 'zipcode_98001', 'zipcode_98002',
       'zipcode_98003', 'zipcode_98004', 'zipcode_98005', 'zipcode_98006',
       'zipcode_98007', 'zipcode_98008', 'zipcode_98010', 'zipcode_98011',
       'zipcode_98014', 'zipcode_98019', 'zipcode_98022', 'zipcode_98023',
       'zipcode_98024', 'zipcode_98027', 'zipcode_98028', 'zipcode_98029',
       'zipcode_98030', 'zipcode_98031', 'zipcode_98032', 'zipcode_98033',
       'zipcode_98034', 'zipcode_98038', 'zipcode_98039', 'zipcode_98040',
       'zipcode_98042', 'zipcode_98045', 'zipcode_98052', 'zipcode_98053',
       'zipcode_98055', 'zipcode_98056', 'zipcode_98058', 'zipcode_98059',
       'zipcode_98065', 'zipcode_98070', 'zipcode_9

In [46]:
# This reorder can be done better in the final set, but since I worked in progression is a remainder of the original work that can be usefoul to test diferent sets in this same code.
# reorder the data to have the target price at the end
column_to_move = new_clean_data4.pop("price")

# insert column with insert(location, column_name, column_value)

new_clean_data4.insert(91, "price", column_to_move)


In [49]:
# get a training set: a X as with the features to relate and a y with the  target 
X = new_clean_data4.iloc[:,0:-1]
y = new_clean_data4.price


In [52]:
# Start spliting the data and import libraries




X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.30, random_state=1)


In [53]:
#  xgboost regresor


# createn instance and fit the model

clf5 = xgboost.XGBRegressor(random_state=5, max_depth= 4, n_estimators= 900, learning_rate= 0.1, verbosity = 0, n_jobs = -1).fit(X_train, y_train)
y_pred = clf5.predict(X_test)

# This is how to generate a regresion report ( for regresion )
print("Mean absolute error in dollars=", round(
    sm.mean_absolute_error(y_test, y_pred), 2))
print("Mean squared error =", round(
    sm.mean_squared_error(y_test, y_pred), 2))
print("Median absolute error =", round(
    sm.median_absolute_error(y_test, y_pred), 2))
print("Explain variance score =", round(
    sm.explained_variance_score(y_test, y_pred), 2))
print("R2 score =", round( sm.r2_score(y_test, y_pred),4))
print( "ROOT Mean squared error in dollars = ", 
    round(sm.mean_squared_error(y_test, y_pred, squared=False 
)))


Mean absolute error in dollars= 62452.45
Mean squared error = 12200338836.37
Median absolute error = 36964.16
Explain variance score = 0.91
R2 score = 0.9085
ROOT Mean squared error in dollars =  110455


In [54]:
# Cross validation of the model with r2 as scoring parameter. Takes at least 1 minute to run all cicles.



scores = cross_val_score(clf5, X, y, scoring='r2', cv=10, n_jobs=-1)
scores.mean()

0.9040149000157249

In [55]:
scores

array([0.89440214, 0.87691611, 0.91461239, 0.91162149, 0.89883774,
       0.91477642, 0.92063524, 0.90115007, 0.9069996 , 0.90019779])